In [ ]:
import pandas as pd     
import numpy as np 
from tabulate import tabulate

In [ ]:
hist23 = pd.read_excel("./initial_data/2023.xlsx")
hist22 = pd.read_excel("./initial_data/2022.xlsx")
litros_board = pd.read_excel("./initial_data/litros_board_22_23.xlsx")

# Raw Data 

Primer intentarem obtenir una sensació de quines columnes contenen les dades històriques i veure si hi ha alguna diferència entre les de 2022 i 2023. Normalitzarem els noms de les columnes i els formats dels valors per unificar els dos històrics en un de uniforme.

In [ ]:
common_columns = hist22.columns.intersection(hist23.columns)

print("Common Columns:", common_columns)

# Find columns that are in df1 but not in df2
only_22 = hist22.columns.difference(hist23.columns)
print("Columns only in historic22:", only_22)

# Find columns that are in df2 but not in df1
only_23 = hist23.columns.difference(hist22.columns)
print("Columns only in historic23:", only_23)

Com bé es pot veure, totes les dades historiques semblen tenir les mateixes columnes i algunes es repeteixen, com Nombre 1 o bé UM. L'historic de 23 te una columna esmentada Número de deudor, la qual es probablement per identificar qui paga la comanda d'aquell bar. Evidentment això no ens es rellevant a nosaltres, aixi que per sinmplificar les coses podriem simplement esborrar per aixi assegurarnos que hi ha les mateixes columnes, a pesar de que s'hauran de normalitzar els valors de totes les columnes perque concordin els 2 historics. A més a més Numero1.1 es el identificador fiscal del bar en concret, pero com estem pre-processant la taula mestre cruda, intentarem deixar el màxim de columnes. També podem esborrar la columna "Tick sin ped" ja que aquesta només conté valors nuls i no aporta cap informació rellevant de totes maneres. Les columnes UM , "N Lote Bolsa" i tota la informació rellevant al document i quan es va crear/modificar el ticket (Es informació logisitica que no ens es útil)

El identificador de barril tampoc ens serveix ja que les dades son incongruents, esborrarem la columna sencera.

In [ ]:
hist23["Tick sin ped"].count() 
hist22["Tick sin ped"].count() 
#Only 5 values are not NaN , therefore this column is a bit useless

useless_columns = ["UM", "UM.1", "Nº Lote Bolsa", "Tick sin ped", "Creado por", "Creado el", "Hora", "Modif.por", "Modif.el", "HoraModif", "ID doc.", "IDContRep", "Tipo doc.", "Nº Tanque"]
hist22.drop(columns=useless_columns, inplace=True)
hist23.drop(columns=useless_columns, inplace=True)
hist23.drop("Número de deudor", inplace=True,axis=1)
print(hist23)
#print(hist23.columns)

## Corrupt Entries (Rows)
Caldria també esborrar les files o entrades que siguin conflictives, que tinguin molts molts NaN's o nulls que no aportin cap mena d'informació al model. Ens adonem que el historic de 2023 te dos files inservibles i també el historic de 2022 te les dues últimes columnes inservibles.

In [ ]:
# Last 2 rows are full of NaN's and nulls, we can remove those entries
#hist23.drop()
print(len(hist23))
print(len(hist22))
hist23 = hist23.drop(hist23.index[-2:])
hist22 = hist22.drop(hist22.index[-2:])
print(hist23)
print(hist22)

#### Les files són numeros de ticket que contenen 1 bar y 1 proveidor que transporta 1 tipus determinat de cervesa d'1 barril en concret. 

Pero primerament volem identificar les claus primàries de les files, en aquest cas semblen ser el numero de ticket. El numero ticket es per cadascun dels barrils que compren comprovem si els seus valors no es repeteixen.

In [ ]:
hist22.rename(columns={   
                          " Nº Ticket": "num_ticket", "Centro Carga": "centro_carga", "Doc.venta": "documento_venta",
                          "Cliente": "id_cliente", "Nombre 1": "desc_cliente","Nombre 1.1": "id_fiscal_proveedor", "Proveedor": "id_proveedor", "EdS": "empresa_proveedor",
                          "Albarán": "id_albaran", "Lote producto": "id_lote_producto", "Sal.mcia.real": "fecha_pedido",
                          "Fecha Caducidad": "fecha_caducidad", "Material": "id_cerveza", "Número de material": "desc_cerveza",
                          "Vehículo": "id_vehiculo", "  CtdAlbFab": "total_litros_transportados", " CtdPedVtas": "litros_pedidos_cliente",
                          "Cantidad entrega": "litros_dados_cliente", "Entrega": "id_orden_entrega",
                          "F.Descarga": "fecha_descarga", "H.ini.Desc": "hora_inicio_descarga", "H.fin.Desc": "hora_fin_descarga",
                          "Presión CO2": "pression_co2", "Temperat.": "temperatura"
                       },
               inplace=True
              )

#hist23 the same adding the column "Nombre1.1"
hist23.rename(columns={   
                          " Nº Ticket": "num_ticket", "Centro Carga": "centro_carga", "Doc.venta": "documento_venta",
                          "Cliente": "id_cliente", "Nombre 1": "desc_cliente","Nombre 1.1": "id_fiscal_proveedor",  "Proveedor": "id_proveedor", "EdS": "empresa_proveedor",
                          "Albarán": "id_albaran", "Lote producto": "id_lote_producto", "Sal.mcia.real": "fecha_pedido",
                          "Fecha Caducidad": "fecha_caducidad", "Material": "id_cerveza", "Número de material": "desc_cerveza",
                          "Vehículo": "id_vehiculo", "  CtdAlbFab": "total_litros_transportados", " CtdPedVtas": "litros_pedidos_cliente",
                          "Cantidad entrega": "litros_dados_cliente", "Entrega": "id_orden_entrega",
                          "F.Descarga": "fecha_descarga", "H.ini.Desc": "hora_inicio_descarga", "H.fin.Desc": "hora_fin_descarga",
                          "Presión CO2": "pression_co2", "Temperat.": "temperatura"
                       },
               inplace=True
              )

#h
mergeable_hist23 = hist23.drop(columns=["id_fiscal_proveedor"])
print(tabulate(hist23, headers="keys", tablefmt="psql"))



In [ ]:
print(tabulate(hist22, headers="keys", tablefmt="psql"))


# Raw Master & column analysis
En aquest apartat crearem una taula mestra que guardarem en un fitxer de format .parquet , ja que aquest a diferencia d'excel o CSV, preserva l'informació dels tipus i noms de les columnes i altres metadates. Fusionarem els històrics de 2022 i 2023. Primerament analitzarem totes les columnes per assegurarnos de la qualitat de les dades

In [ ]:
print(f"first ticket {hist22['num_ticket'].min()}, last ticket {hist22['num_ticket'].max()}")
print(f"first ticket {hist23['num_ticket'].min()}, last ticket {hist23['num_ticket'].max()}")


## Re-cast Data Types 
Abans de poder guardar les taules en fitxers permanents, haurem de cambiar els tipus a tipus bàsics com strings, enters i altres. Caldrà tractar els valors nuls de dues columnes que no permeten la conversió a tipus int (degut a NaN's i infinits) i usar regex a la columna de temperatura per evitar que les comes ', ens impedeixen la lectura a floats de la temperatura.

In [ ]:
#print(hist22.dtypes)

hist22["documento_venta"] = hist22["documento_venta"].fillna(0).astype(int)
hist22["id_orden_entrega"] = hist22["id_orden_entrega"].fillna(0).astype(int)
hist22["temperatura"] = hist22["temperatura"].replace("'", ".", regex=True)

hist23["documento_venta"] = hist23["documento_venta"].fillna(0).astype(int)
hist23["id_orden_entrega"] = hist23["id_orden_entrega"].fillna(0).astype(int)
hist23["temperatura"] = hist23["temperatura"].replace("'", ".", regex=True)



type_coertion_dict = {
    "num_ticket": int,
    "centro_carga": int,
    "documento_venta": int,
    "id_cliente": int, 
    "desc_cliente": str,
    "id_proveedor": int,
    "id_fiscal_proveedor": str,
    "empresa_proveedor": str,
    "id_albaran": int,
    "id_lote_producto": str,
    "fecha_pedido": str,
    "fecha_caducidad": str,
    "id_cerveza": str,
    "desc_cerveza": str, 
    "id_vehiculo": str,
    "id_orden_entrega": int,
    "fecha_descarga": str,
    "hora_inicio_descarga": str,
    "hora_fin_descarga": str,
    "temperatura": float
}
hist22 = hist22.astype(type_coertion_dict)
hist23 = hist23.astype(type_coertion_dict)
#print(hist23.dtypes)

In [ ]:
hist22.to_parquet("./processed_data/1_raw_historical_2022.parquet", index=True)
#print(hist22.dtypes)
hist23.to_parquet("./processed_data/1_raw_historical_2023.parquet", index=True)

master_historical = pd.concat([hist22, hist23], ignore_index=True, sort=True)
print(tabulate(master_historical, headers="keys", tablefmt="psql"))
#print(master_historical["num_ticket"].describe())
master_historical.to_parquet("./processed_data/1_raw_master.parquet",index=True)
